# 8장. 생성 모델을 위한 딥러닝(Deep Learning for generative models)

* 이 장에서 다룰 핵심 내용
    * LSTM을 사용할 텍스트 생성
    * 딥드림 구현
    * 뉴럴 스타일 트랜스퍼 구현
    * 변이형 오토인코더
    * 적대적 생성 네트워크 이해

* 2015년 여름 구글의 딥드림 알고리즘이 이미지를 강아지 눈과 환상적인 인공물이 뒤섞인 몽환적인 그림으로 바꾸어 많은 사람들의 흥미를 끌었습니다. 2016년에는 프리즈마(Prizma) 애플리케이션이 사람들의 사진을 다양한 스타일의 그림으로 바꾸어 주었습니다. 2016년 여름 실험적인 단편 영화 <**썬스프링**(Sunspring)>은 LSTM 알고리즘이 쓴 완전한 대화가 포함된 각본을 사용하여 만들어졌습니다. 아마도 신경망이 시험 삼아 작곡한 음악도 최근에 들어 보았을 것입니다.
* 사실 지금까지 본 AI 예술 작품의 품질은 낮습니다. AI가 실제 작가나 화가, 작곡가와 견줄 수준은 못 됩니다. 사람을 대신한다는 것은 늘 핵심을 벗어난 생각입니다.인공 지능이 사람의 지능을 어떤 다른 것으로 대체하지 않습니다. 인공 지능은 우리 생활과 일에 지능을 더합니다. 이 지능은 다른 종류의 지능입니다. 여러 분야에서 특히 예술에서는 AI가 사람의 능력을 증가시키는 도구로 사용될 것입니다. 즉 인공적인 지능이 아니라 **확장된 지능**(augmented intelligence)입니다.
* 예술 창작의 대부분은 간단한 패턴 인식과 기술로 만들어집니다. 많은 사람은 이런 과정에 관심이 없거나 심지어 불필요하다고 생각하기도 합니다. 여기에 AI가 필요합니다. 사람의 지각, 언어, 예술 작품은 모두 통계적 구조를 가집니다. 딥러닝 알고리즘은 이 구조를 학습하는 데 뛰어납니다. 머신 러닝 모델은 이미지, 음악, 글의 통계적 **잠재 공간**(latent space)을 학습할 수 있습니다. 그다음 이 공간에서 샘플을 뽑아 새로운 예술 작품을 만들 수 있습니다. 이 작품은 모델이 훈련 데이터에서 본 것과 비슷한 특징을 가질 것입니다. 당연히 이런 샘플링 자체가 예술 창작이라고 보기는 어렵습니다. 사실 수학 연산에 불과합니다. 알고리즘은 사람의 삶, 감정, 세성에 대한 경험이 없습니다. 사람과 매우 다른 경험에서 배웁니다. 이는 단지 제3자의 입장에서 알고리즘이 만든 것의 의미를 해석한 것입니다. 숙련된 예술가가 사용하면 알고리즘 창작은 의미 있고 아름다운 것으로 바뀔 수 있습니다. 잠재 공간 샘플링은 예술가의 능력을 높이는 붓이 될 수 있습니다. 또 창작 가능성을 늘리며 상상의 공간을 확장시킵니다. 이외에도 기교와 훈련이 필요하지 않기 때문에 예술 창작을 더욱 자유롭게 만들어 줍니다. 기교에서 예술을 분리하여 순수한 표현을 만드는 새로운 도구입니다.
* 전자 음악과 컴퓨터 음악의 선구자인 이안니스 크세나키스(Iannis Xenakis)는 1960년대에 자동 작곡 애플리케이션에 대해 동일한 아이디어를 아름답게 표현했습니다.
    * 작곡가가 지루한 작업에서 벗어나면 새로운 음악 형식에 관한 일반적인 문제에 집중하고 입력 데이터를 수정하면서 이 형식의 구석구석을 탐험할 수 있습니다. 예를 들어 독주에서 실내 악단, 대형 오케스트라까지 모든 악기의 조합을 테스트할 수 있습니다. 컴퓨터를 활용하는 작곡가는 일종의 파일럿입니다. 출발 버튼을 누르고 좌표를 설정하고 소리의 우주를 항해하는 우주선을 조정합니다. 이전에는 머나먼 꿈처럼 생각했던 음악의 별자리와 은하 사이를 여행하게 됩니다.
* 이 장에서 예술 창작에 딥러닝이 어떻게 쓰일 수 있는지 다양한 각도에서 살펴보겠습니다. 시퀀스 데이터 생성(글을 쓰거나 작곡할 수 있습니다)과 딥드림, 변이형 오토인코더, 적대적 생성 네트워크(generative adversarial network)를 사용한 이미지 생성을 알아보겠습니다. 컴퓨터가 새로운 꿈을 꾸도록 만들어 보죠. 어쩌면 우리도 꿈을 꾸게 될지 모릅니다. 기술과 예술의 접점에 놓인 놀라운 가능성에 관한 꿈입니다.

## 8.1 LSTM으로 텍스트 생성하기

* 이 절에서 순환 신경망으로 시퀀스 데이터를 생성하는 방법을 살펴보겠습니다. 텍스트 생성을 예로 들지만 동일한 기법으로 어떤 종류의 시퀀스 데이터도 생성할 수 있습니다. 음표의 시퀀스에 적용하여 새로운 음악을 만들거나 연속된 붓질 시퀀스(예를 들어 화가가 아이패드에 그림을 그리는 과정을 기록한 것)에 적용하여 한 획 한 획 그림을 그릴 수 있습니다.
* 시퀀스 데이터 생성이 예술 콘텐츠에 국한되지 않습니다. 음성 합성과 챗봇의 대화 기능에 성공적으로 적용되었습니다. 구글이 2016년에 공개한 스마트 답장(Smart Reply)도 비슷한 기술을 사용합니다. 짧은 문장을 자동으로 생성하여 이메일이나 문자 메시지로 답장을 보낼 수 있습니다.

### 8.1.1 생성 RNN(Recurrent Neural Network 순환 신경망)의 간단한 역사


* 2014년 후반 머신 러닝 공동체에서도 소수의 사람들만이 LSTM이란 용어를 알았습니다. 순환 네트워크(RNN)를 사용하여 시퀀스 데이터(Sequence data)를 성공적으로 생성한 애플리케이션은 2016년이 되어서야 주류가 되기 시작했습니다.하지만 이 기술은 1997년 LSTM 알고리즘이 개발된 이래 역사가 꽤 오래 되었습니다. 이 알고리즘은 초기에 글자를 하나씩 생성하는 데 사용되었습니다.
* 2002년 더글라스 에크(Douglas Eck)는 스위스의 슈미드후버(Schmidhuber)의 연구실에서 LSTM을 음악 생성에 처음 적용하여 가능성 있는 결과를 얻었습니다. 에크는 현재 구글 브레인의 연구원으로, 2016년에는 마젠타(Magenta)란 새로운 연구 그룹을 시작했습니다. 여기에서는 최신 딥러닝 기술을 사용하여 멋진 음악을 만드는 것에 집중하고 있습니다. 때로는 좋은 아이디어가 실현되는데 15년이 걸리기도 합니다.
* 2000년대 후반과 2010년대 초반에 알렉스 그레이브스(Alex Graves)는 순환 네트워크를 사용하여 시퀀스 데이터를 생성하는 데 아주 중요한 선구적인 일을 했습니다. 특히 2013년에 펜 위치를 기록한 시계열 데이터를 사용하여 순환 네트워크와 완전 연결 네트워크를 혼합한 네트워크로 사람이 쓴 것 같은 손글씨를 생성했으며, 이 작업이 전환점이 되었습니다. 때맞추어 등장한 특별한 이 신경망 애플리케이션은 꿈을 꾸는 컴퓨터를 상상하게 만들었고, 필자가 케라스를 개발할 때 많은 영감을 주었습니다. 그레이브스는 논문 사전 출판 서버인 아카이브(arXiv)에 업로드한 2013년 레이텍(LaTex) 파일에 비슷한 주석을 남겼습니다. "시퀀스 데이터를 생성하는 것은 컴퓨터가 꿈을 꾸게 하는 것입니다." 몇 년이 지난 지금 우리는 이런 애플리케이션을 당연하게 받아들입니다. 그 당시에는 그레이브스의 실험을 보고 가능성만으로 대단한 것이라고 생각하기는 어려웠습니다.  
* 그 이후 순환 신경망은 음악 생성, 대화 생성, 이미지 생성, 음성 합성, 분자 설계(molecule design)에 성공적으로 사용되었습니다. 심지어 실제 배우가 캐스팅된 영화의 대본을 만드는 데도 사용되었습니다.

### 8.1.2 시퀀스 데이터를 어떻게 생성할까?
* 딥러닝에서 시퀀스 데이터를 생성하는 일반적인 방법은 이전 토큰을 입력으로 사용해서 시퀀스의 다음 1개 또는 몇 개의 토큰을 (RNN이나 컨브넷(convnet)으로) 예측하는 것입니다. 예를 들어 "the cat is on ma"란 입력이 주어지면 다음 글자인 타깃 "t"를 예측하도록 네트워크를 훈련합니다. 텍스트 데이터를 다룰 때 토큰은 보통 단어 또는 글자입니다. 이전 토큰들이 주어졌을 때 다음 토큰의 확률을 모델링할 수 있는 네트워크를 **언어 모델**(language model)이라고 부릅니다. 언어 모델은 언어의 통계적 구조인 잠재 공간을 탐색합니다.
* 언어 모델을 훈련하고 나면 이 모델에서 샘플링을 할 수 있습니다(새로운 시퀀스를 생성합니다). 초기 텍스트 문자열을 주입하고(**조건 데이터**(conditioning data)라고 부릅니다) 새로운 글자나 단어를 생성합니다(한 번에 여러 개의 토큰을 생성할 수도 있습니다). 생성된 출력은 다시 입력 데이터로 추가됩니다. 이 과정을 여러 번 반복합니다. 이런 반복을 통해 모델이 훈련한 데이터 구조가 반영된 임의의 길이를 가진 시퀀스를 생성할 수 있습니다. 사람이 쓴 문장과 거의 비슷한 시퀀스를 만듭니다. 이 절의 예제는 LSTM층을 사용합니다. 텍스트 말뭉치(corpus)에서 N개의 글자로 이루어진 문자열을 추출하여 주입하고 N + 1번째 글자를 예측하도록 훈련합니다. 모델의 출력은 출력 가능한 모든 글자에 해당하는 소프트맥스(softmax) 값입니다. 즉 다음 글자의 확률 분포입니다. 이 LSTM을 **글자 수준의 신경망 언어 모델**(character-level neural language model)이라고 부릅니다.

### 8.1.3 샘플링 전략의 중요성
* 텍스트를 생성할 때 다음 글자를 선택하는 방법이 아주 중요합니다. 단순한 방법은 항상 가장 높은 확률을 가진 글자를 선택하는 **탐욕적 샘플링**(greedy sampling)입니다. 이 방법은 반복적이고 예상 가능한 문자열을 만들기 때문에 논리적인 언어처럼 보이지 않습니다. 좀 더 흥미로운 방법을 사용하면 놀라운 선택이 만들어집니다. 다음 글자의 확률 분포에서 샘플링하는 과정에 무작위성을 주입하는 방법입니다. 이를 **확률적 샘플링**(stochastic sampling)이라고 부릅니다(머신 러닝에서 확률적(stochastic)이란 뜻은 무작위(random)하다는 뜻입니다). 이런 방식을 사용할 경우 'e'가 다음 글자가 될 확률이 0.3 이라면, 모델이 30%정도는 이 글자를 선택합니다. 탐욕적 샘플링을 확률적 샘플링으로 설명할 수도 있습니다. 한 글자의 확률이 1이고 나머지 글자는 모두 0인 확률 분포를 가지는 경우입니다.
* 모델의 소프트맥스 출력은 확률적 샘플링에 사용하기 좋습니다. 이따금 샘플링될 것 같지 않은 글자를 샘플링합니다. 훈련 데이터에는 없지만 실제 같은 새로운 단어를 만들어 재미있고 창의적으로 보이는 문장을 생성합니다. 이 전략에는 한 가지 문제가 있습니다. 샘플링 과정에서 무작위성의 양을 조절할 방법이 없습니다.
* 왜 무작위성이 크거나 작아야 할까요? 극단적인 경우를 생각해 보죠. 균등 확률 분포에서 다음 글자를 추출하는 완전한 무작위 샘플링이 있습니다. 모든 글자의 확률은 같습니다. 이 구조는 무작위성이 최대입니다. 다른 말로 하면 이 확률 분포는 최대의 엔트로피를 가집니다. 당연하게 흥미로운 것들을 생산하지 못합니다. 반대의 경우 탐욕적 샘플링은 무작위성이 없기 때문에 흥미로운 것을 전혀 만들지 못합니다. 탐욕적 샘플링의 확률 분포는 최소의 엔트로피를 가집니다. 모델의 소프트맥스 출력인 '실제' 확률 분포에서 샘플링하는 것은 이 두 극단의 중간에 위치해 있습니다. 중간 지점에는 시도해 볼 만한 더 높거나 낮은 엔트로피가 많습니다. 작은 엔트로피는 예상 가능한 구조를 가진 시퀀스를 생성합니다(더 실제처럼 보입니다). 반면에 높은 엔트로피는 놀랍고 창의적인 시퀀스를 만듭니다. 생성 모델에서 샘플링을 할 때 생성 과정에 무작위성의 양을 바꾸어 시도해 보는 것이 좋습니다. 흥미는 매우 주관적이므로 최적의 엔트로피 값을 미리 알 수 없기 때문입니다. 얼마나 흥미로운 데이터를 생성할 것인지는 결국 사람이 판단해야 합니다.
* 샘플링 과정에서 확률의 양을 조절하기 위해 **소프트맥스 온도**(softmax temperature)라는 파라미터를 사용합니다. 이 파라미터는 샘플링에 사용되는 확률 분포의 엔트로피를 나타냅니다. 얼마나 놀라운 또는 예상되는 글자를 선택할 지 결정합니다. temperature 값이 주어지면 다음과 같이 가중치를 적용하여 (모델의 소프트맥스 출력인) 원본 확률 분포에서 새로운 확률 분포를 계산합니다.

In [ ]:
# 다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기
# Replace the weight of a probability distribution by using different temperature values
import numpy as np

# original distribution은 전체 합이 1인 1D 넘파일 배열입니다.
# temperature는 출력 분포의 엔트로피 양을 결정합니다.
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    # 원본 분포의 가중치를 변경하여 반환합니다.
    # 이 분포의 합은 1이 아닐 수 있으므로 새로운 분포의 합으로 나눕니다.
    return distribution / np.sum(distribution)

* 높은 온도는 엔트로피가 높은 샘플링 분포를 만들어 더 놀랍고 생소한 데이터를 생성합니다. 반면에 낮은 온도는 무작위성이 낮기 때문에 예상할 수 있는 데이터를 생산합니다.

★ LSTM으로 텍스트 생성하기(Text generation with LSTM)

### 8.1.4 글자 수준의 LSTM 텍스트 생성 모델 구현
* 이런 아이디어를 케라스로 구현해 보죠. 먼저 언어 모델을 학습하기 위해 많은 텍스트 데이터가 필요합니다. 위키피디아나 반지의 제왕처럼 아주 큰 텍스트 파일이나 텍스트 파일의 묶음을 사용할 수 있습니다. 이 예에서는 19세기 후반 독일의 철학자 니체(Nietzsche)의 글을 사용하겠습니다(영어로 번역된 글입니다). 학습할 언어 모델은 일반적인 영어 모델이 아니라 니체의 문체와 특정 주제를 따르는 모델일 것입니다.

#### 데이터 전처리
* 먼저 말뭉치를 다운로드하고 소문자로 바꿉니다.

In [3]:
# 원본 텍스트 파일을 내려받아 파싱하기
# Download and parse the original text file
import tensorflow.keras
import numpy as np

path = tensorflow.keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('말뭉치 크기:', len(text))

606208/600901 [==============================] - 1s 2us/step
말뭉치 크기: 600893


In [4]:
type(text)

str

* 그 다음 `maxlen` 길이를 가진 시퀀스를 중복하여 추출합니다. 추출된 시퀀스를 원-핫 인코딩으로 변환하고 크기가 `(sequences, maxlen, unique_characters)`인 3D 넘파이 배열 `x`로 합칩니다. 동시에 훈련 샘플에 상응하는 타깃을 담은 배열 `y`를 준비합니다. 타깃은 추출된 시퀀스 다음에 오는 원-핫 인코딩된 글자입니다.

In [5]:
# 글자 시퀀스 벡터화하기
# Vectorize the character sequence

# 60개 글자로 된 시퀀스를 추출합니다.
maxlen = 60

# 세 글자씩 건너 뛰면서 새로운 시퀀스를 샘플링합니다.
step = 3

# 추출한 시퀀스를 담을 리스트
sentences = []

# 타깃(시퀀스 다음 글자)을 담을 리스트
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('시퀀스 개수:', len(sentences))

# 말뭉치에서 고유한 글자를 담은 리스트
chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 200278
고유한 글자: 58
벡터화...


#### 네트워크 구성
* 이 네트워크는 하나의 `LSTM` 층과 그 뒤에 `Dense` 분류기가 뒤따릅니다. 분류기는 가능한 모든 글자에 대한 소프트맥스 출력을 만듭니다. 순환 신경망이 시퀀스 데이터를 생성하는 유일한 방법은 아닙니다. 최근에는 1D 컨브넷도 이런 작업에 아주 잘 들어 맞는다는 것이 밝혀졌습니다.

In [8]:
# 다음 글자를 예측하기 위한 단일 LSTM 모델 만들기
# create single LSTM model to predict the next character
from tensorflow.keras import layers

model = tensorflow.keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

* 타깃이 원-핫 인코딩되어 있기 때문에 모델을 훈련하기 위해 `categorical_crossentropy` 손실을 사용합니다.

In [9]:
# 모델 컴파일 설정하기
# set up model compile
optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

c:\users\wnstj\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


#### 언어 모델 훈련과 샘플링(Training language modle and sampling)

* 훈련된 모델과 시드로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있습니다.

    1.	지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다.
    2.	특정 온도로 이 확률 분포의 가중치를 조정합니다.
    3.	가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
    4.	새로운 글자를 생성된 텍스트의 끝에 추가합니다.

* 다음 코드는 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출합니다(샘플링 함수입니다).

In [10]:
# 모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수 구현하기
# implement Function to sample new characters with given the model's prediction
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

* 마지막으로 다음 반복문은 반복적으로 훈련하고 텍스트를 생성합니다. 에포크마다 학습이 끝난 후 여러가지 온도를 사용해 텍스트를 생성합니다. 이렇게 하면 모델이 수렴하면서 생성된 텍스트가 어떻게 진화하는지 볼 수 있습니다. 온도가 샘플링 전략에 미치는 영향도 보여 줍니다.

In [ ]:
# 텍스트 생성 루프 만들기
# create text generation loop
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

# 60 에포크 동안 모델을 훈련합니다
for epoch in range(1, 60):
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size=128, epochs=1)

    # 무작위로 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도합니다
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
        for i in range(400):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
1565/1565 [==============================] - 360s 228ms/step - loss: 1.9605
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the more the compertal and there is the more something the is the completed the great the more there is the completely and the the serious and all the been and the one stand and the the the in the same there is been the more been of the been the inderitions of the good the experience, there is a proterst the as the been the something the experience, in the sears and the sears of the preations of
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the fares of the beer one the as it is it as there is for a perhaps it is the conderstion of a beenst there in the become as the great the preas the sasters there is has one many in the belistations, and to been so there is the mame to the sething there is a perhaps of the communice and st

middain thio may rose. "h kere case of counted on to doon that spires, proper-founally morals posstinds), or sebst to highers. vhels: thore are timehgre ss
에포크 5
1565/1565 [==============================] - 327s 209ms/step - loss: 1.4503
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the soul of the soul is the present to be all the believed to the self-contrary in the sense of the man is not to the sense of the soul--the moral rational form of the same that the schopenhauer of the same to the believe in the antich to the same the antain to say the soul and soul of the schopenhauer, and are profounders of the man and profound in the soul of the sense of the sense of the soul
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for a present string
in the other in the whole the laver of the nimble to say and happension of the same soul--the conscience of the stron

human under the ake. the mort and na)krtable
until, 'domatige, alougviesvity. constrationskope in-ashured. if--hejoallies.

the a little allate? only friends--what platonical, aiddibild for this sove ver? t
에포크 9
1565/1565 [==============================] - 331s 211ms/step - loss: 1.3870
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the same the world of the sense of the same in the consequence of the present and stronger of the same the same soul of the consequence of the same soul and the soul and soul and the age of the world and the same the stronger of the sense to the same the more proparity of the sense to the same the sense to the same the same the sense to the stronger of the same soul, and in the same stronger and
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the propager world and the delight in comparation to the spirit to the indeptheby u

waves; as religionally; throuthous ranes is only traated cowern is perslysness,
society,ing
fir instance of emition"s
bors, be for. learl,, but did turnih, system" dicise for
thpible immidltates "degrady. we fow, forield. to foundsionibles of moralities, so philosopher
for createsing, loves any
에포크 13
1565/1565 [==============================] - 357s 228ms/step - loss: 1.3549
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the strength to the same the fact that the suffering the conceals the profound and the strength the strength to the same to be allow the same the same the strength the proper the strength the conceive of the same the strength and the soul, and with religion is the same the same the strength the strength the commence of the sense of the same the proper the strength that the sense of the same the 
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
throu

through forwmends, human, that a loves before in different, hren only ouh only to layed to which very
coo jueicing
of rognished a the lest share about, as brickes--agreested labous of
such
tle,
heart werament,
as art, had now demitic power: destright, exjoy devine".)---his cheinens
of head onw
one
adrse,
deduponmans werl
shanss, and with good stradigalating does not gethine of folthoc... "we is will oncesim
에포크 17
1565/1565 [==============================] - 320s 205ms/step - loss: 1.3344
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the command of the same themselves and command in the same the present one is always the same themselves the same themselves because it is always the manifest themselves and stronger of the sense of the desire the religious also all themselves in the same as the same themselves the relation of the same themselves and the most self-comprehendent it is a

...

ble of cobrit, hard
fiouge "force.


11

=nisk" and frness-stensable? and has always
sye of cha=nming
presentigual still the rarsones himself like the unsires! which they allow, even a
go dagen daze.

      ord.lyipline
roped the ervius, m
에포크 19
1565/1565 [==============================] - 332s 212ms/step - loss: 1.3258
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the same than the consideration of the same measured to many of the present dear that the feelings of the consideration of the same time and proper the consequences of the same trans of the same dealing and soul and consequences of the same trans the self-consequences of the same true and still and soul and present distinguished to the fact that the same the consequences of the same the state of
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for which in the most soul, and the experience, and a

through formule in constancisy of a nivelrwomess in the variest worring perhan; it be anwaves. wher makes its grimpos" of the eithenf of world
rooo.

l]jyis
sprike ye demand.tice toners as the ouvablewer staten, in the briefly belong to be instincts its purpose as cany of ear flore
sayo."
in euroups of duties are to making then-realordity, also a delightes, it."

involor destsumppine personts", should even 
에포크 23
1565/1565 [==============================] - 333s 213ms/step - loss: 1.3124
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the sense of the same as the profound the world of the same and sense of the fact that the man with the same and the conscience of the same as a moral and man and self-consistions and sense of the same as a great souls, and in the same as a man with the same as a person of the conscience and sense of the extent to the greatest and sense of the same and

through found more horee, to,
luthertsd, and foreger fundamental
men of a chonces. the apart that an objectiinly through a made, so much or an alter, as aristment hembutt
from their meprahing visished with a grediters,--nets does this coniditied has
ipinto vircly remoset in cra, what
sprinting is justof formet, to mactimu and bstsat the type vindly any?!--conduct a general
instincts, baso" truth, overrew, h
에포크 27
1565/1565 [==============================] - 353s 226ms/step - loss: 1.3018
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through formerly that the fact the same truth, and all such a person with the same trated and spirit of the strongess of the fact that the spirit to be an action of the consequently the most faith to the most exists and the spirit and the consequently and spirit, the consequently and the property to be a desire the most spirit to perhaps be and the feelings of the

through for a -citen mistakenness, when out.
merely
eliadent called prutties clo-nowness of ever not at present through
.omat soblences-loves of aweder a good--late, "the. the sight, forth in his belief, thmegwans which good
ignoble sircally first
melade, a
sought man is also
impossial namegogics, but he, that are, that what
day
henthe its (whick; "this also, truth, the usw even of self-meptaries," any-god 
에포크 31
1565/1565 [==============================] - 346s 221ms/step - loss: 1.2909
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the more as the contrary and self-contradict
of a more and the soul and the same transcted the condition of the same transcends of the sense of the states of the marks of the sensition of the problem of the art of the same trained of the sensition of the sense of the same and the desire of the sense of the same and the really the more all the sense of 

through for thought.

127. thas gowcey, bloryboula
never, the acts writish of them progles of truth: to all pogess is very view. as the closely onigic 
high. if a notglism of
the unconditionaly, and has protection and vagianic nature, this was falsiality," that has at these o
crumvely. as
our same troictive, but were a has neyosnful, of worthived that
is dodsol as i arorden, to pels
tkeent humanity,
astipis
에포크 35
1565/1565 [==============================] - 347s 222ms/step - loss: 1.2841
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the consequence of the spirit to the same and with the same truth and the sense of the same the most the same consider to the faith and consider the present the most and consider the most contempt the spirit and second the present in the world to the same to the same as the consequence of the spirit of the same to the same to the wor the fact that the 

* 여기서 볼 수 있듯이 낮은 온도는 아주 반복적이고 예상되는 텍스트를 만듭니다. 하지만 국부적인 구조는 매우 실제와 같습니다. 특히 모든 단어(단어는 글자의 지역 패턴으로 이루어집니다)가 실제 영어 단어입니다. 높은 온도에서 생성된 텍스트는 아주 흥미롭고 놀라우며 창의적이기도 합니다. 이따금 꽤 그럴싸하게 보이는 완전히 새로운 단어를 창조합니다(‘begarmed’와 ‘isharent’ 같은 단어입니다). 높은 온도에서는 국부적인 구조가 무너지기 시작합니다. 대부분의 단어가 어느정도 무작위한 문자열로 보입니다. 확실히 이 네트워크에서는 텍스트 생성에 가장 좋은 온도는 0.5입니다. 항상 다양한 샘플링 전략으로 실험해 봐야합니다! 학습된 구조와 무작위성 사이에 균형을 잘 맞추면 흥미로운 것을 만들 수 있습니다.

* 더 많은 데이터에서 크고 깊은 모델을 훈련하면 이것보다 훨씬 논리적이고 실제와 같은 텍스트 샘플을 생성할 수 있습니다. 당연히 우연이 아닌 의미 있는 텍스트가 생성된다고 기대하지 마세요. 글자를 연속해서 나열하기 위한 통계 모델에서 데이터를 샘플링한 것뿐입니다. 언어는 의사소통의 수단입니다. 의사소통이 의미하는 것과 의사소통이 인코딩된 메시지의 통계 구조 사이는 차이가 있습니다. 이 차이를 검증하기 위해 다음과 같은 사고 실험을 해보죠. 컴퓨터가 대부분의 디지털 통신에서 하는 것처럼 사람의 언어가 의사소통을 압축하는데 더 뛰어나다면 어떨까요? 언어의 의미가 줄진 않지만 고유한 통계 구조가 사라질 것입니다. 이는 방금과 같은 언어 모델을 학습하는 것을 불가능하게 만듭니다.

### 8.1.5 정리

* 이전의 토큰이 주어지면 다음 토큰(들)을 예측하는 모델을 훈련하여 시퀀스 데이터를 생성할 수 있습니다.
* 텍스트의 경우 이런 모델을 **언어 모델**이라 부릅니다. 단어 또는 글자 단위 모두 가능합니다.
* 다음 토큰을 샘플링할 때 모델이 만든 출력에 집중하는 것과 무작위성을 주입하는 것 사이에 균형을 맞추어야 합니다.
* 이를 위해 소프트맥스 온도 개념을 사용합니다. 항상 다양한 온도를 실험해서 적절한 값을 찾습니다.

## 8.2 딥드림(DeepDream)
* **딥드림**(DeepDream)은 합성곱 신경망이 학습한 표현을 사용하여 예술적으로 이미지를 조작하는 기법입니다. 2015년 여름 구글이 카페(Caffe) 딥러닝 라이브러리를 사용하여 구현한 것을 처음 공개했습니다(텐서플로가 공개되기 몇 달 전입니다). 딥드림이 생성한 몽환적인 사진은 순식간에 인터넷에 센세이션을 일으켰습니다. 알고리즘으로 변경된 환상적인 인공물, 새 깃털, 강아지 눈이 가득 차 있습니다. 이 딥드림은 다양한 종류의 강아지와 새가 있는 ImageNet 데이터셋에서 훈련된 컨브넷을 사용했습니다.

* 딥드림 알고리즘은 5장에서 소개한 컨브넷을 거꾸로 실행하는 컨브넷 필터 시각화 기법과 거의 동일합니다. 컨브넷 상위 층에 있는 특정 필터의 활성화를 극대화하기 위해 컨브넷의 입력에 경사 상승법을 적용했습니다. 몇 개의 사소한 차이를 빼면 딥드림도 동일한 아이디어를 사용합니다.
    * 딥드림에서는 특정 필터가 아니라 전체 층의 활성화를 최대화합니다. 한꺼번에 많은 특성을 섞어 시각화합니다.
    * 빈 이미지나 노이즈가 조금 있는 입력이 아니라 이미 가지고 있는 이미지를 사용합니다. 그 결과 기존 시각 패턴을 바탕으로 이미지의 요소들을 다소 예술적인 스타일로 왜곡시킵니다.
    * 입력 이미지는 시각 품질을 높이기 위해 여러 다른 스케일(**옥타브** (octave))로 처리합니다.
* 그럼 딥드림 이미지를 만들어 봅시다.

### 8.2.1 케라스 딥드림 구현
* ImageNet에서 훈련한 컨브넷을 가지고 시작하겠습니다. 케라스에는 이렇게 사용할 수 있는 컨브넷이 많습니다. VGG16, VGG19, Xception, ResNet50 등입니다. 이 중에 어느 것을 사용해도 딥드림을 구현할 수 있습니다. 당연히 어떤 컨브넷을 선택했느냐에 따라 시각화에 영향을 미칩니다. 각 컨브넷 구조가 학습한 특성이 다르기 때문입니다. 원래 딥드림에서 사용한 컨브넷은 인셉션 모델입니다. 실제로 인셉션이 멋진 딥드림 이미지를 잘 만듭니다. 여기에서도 케라스의 인셉션 V3 모델을 사용하겠습니다.

In [ ]:
# 사전 훈련된 인셉션 V3 모델 로드하기
# Load pretrained inception V3 model 
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K

# 모델을 훈련하지 않습니다. 이 명령은 모든 훈련 연산을 비활성화합니다
K.set_learning_phase(0)

# 합성곱 기반층만 사용한 인셉션 V3 네트워크를 만듭니다. 사전 훈련된 ImageNet 가중치와 함께 모델을 로드합니다
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

* 그 다음 손실을 계산합니다. 경사 상승법으로 최대화할 값입니다. 5장 필터 시각화에서 특정 층의 필터 값을 최대화했습니다. 여기에서는 여러 층에 있는 모든 필터 활성화를 동시에 최대화합니다. 특별히 상위 층에 있는 활성화의 L2 노름에 대한 가중치 합을 최대화하겠습니다. 정확히 어떤 층들을 선택했는지에 따라 (당연히 최종 손실에 기여한 정도에 따라) 만들어 내는 시각 요소에 큰 영향을 미칩니다. 어떤 층을 선택할지 파라미터로 손쉽게 바꿀 수 있어야 좋습니다. 하위 층은 기하학적인 패턴을 만들고 상위 층은 ImageNet에 있는 클래스로 보이는 시각 요소를 만듭니다(예를 들어 새나 강아지). 먼저 임의로 네 개의 층을 선택해 보겠습니다. 나중에 다른 설정을 다양하게 시도해 보는 것이 좋습니다.

In [ ]:
# 딥드림 설정하기(set up DeepDream)
# 층 이름과 계수를 매핑한 딕셔너리.
# 최대화하려는 손실에 층의 활성화가 기여할 양을 정합니다.
# 층 이름은 내장된 인셉션 V3 애플리케이션에 하드코딩되어 있는 것입니다.
# model.summary()를 사용하면 모든 층 이름을 확인할 수 있습니다
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

* 이제 손실 텐서를 정의하겠습니다. 위에서 선택한 층의 활성화에 대한 L2 노름의 가중치 합입니다.

In [ ]:
# 최대화할 손실 정의하기
# Defining loss to maximize
# 층 이름과 층 객체를 매핑한 딕셔너리를 만듭니다.
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# 손실을 정의하고 각 층의 기여분을 이 스칼라 변수에 추가할 것입니다
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    # 층의 출력을 얻습니다
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    # 층 특성의 L2 노름의 제곱을 손실에 추가합니다. 이미지 테두리는 제외하고 손실에 추가합니다.
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

* 그 다음 경사 상승법 과정을 준비합니다.

In [ ]:
# 경사 상승법 과정(Gradient Ascending Process)
# 이 텐서는 생성된 딥드림 이미지를 저장합니다
dream = model.input

# 손실에 대한 딥드림 이미지의 그래디언트를 계산합니다
grads = K.gradients(loss, dream)[0]

# 그래디언트를 정규화합니다(이 기교가 중요합니다)
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# 주어진 입력 이미지에서 손실과 그래디언트 값을 계산할 케라스 Function 객체를 만듭니다
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

# 이 함수는 경사 상승법을 여러 번 반복하여 수행합니다
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i, '번째 손실 :', loss_value)
        x += step * grad_values
    return x

* 마지막으로 진짜 딥드림 알고리즘입니다.

* 먼저 이미지를 처리하기 위한 스케일(옥타브라고도 부릅니다) 리스트를 정의합니다. 스케일은 이전 스케일보다 1.4배 큽니다(40% 증가합니다). 작은 이미지로 시작해서 점점 크기를 키웁니다.

![deep dream process](https://s3.amazonaws.com/book.keras.io/img/ch8/deepdream_process.png)

* 가장 작은 것에서 가장 큰 스케일까지 연속적인 각 단계에서 정의한 손실이 최대화되도록 경사 상승법을 수행합니다. 경상 상승법이 실행된 후 이미지 크기를 40% 증가시킵니다.

* 스케일을 연속적으로 증가시키면서 (점점 뭉개지거나 픽셀 경계가 나타나므로) 이미지 상세를 많이 잃지 않도록 간단한 기교를 사용합니다. 스케일을 늘린 후 이미지에 손실된 디테일을 재주입합니다. 원본 이미지가 크기를 늘렸을 때 어땠는지 알기 때문에 가능합니다. 작은 이미지 크기 S와 큰 이미지 크기 L이 주어지면 크기 L로 변경된 원본 이미지와 크기 S로 변경된 원본 이미지 사이의 차이를 계산합니다. 이 차이가 S에서 L로 변경되었을 때 잃어버린 디테일입니다.

In [ ]:
# 연속적인 스케일에 걸쳐 경사 상승법 실행하기
# Execute gradient ascending across a continuous scale
import numpy as np
# 하이퍼파라미터를 바꾸면 새로운 효과가 만들어집니다.
# 경사 상승법 단계 크기
step = 0.01
# 경사 상승법을 실행할 스케일 단계 횟수
num_octave = 3
# 스케일 간의 크기 비율
octave_scale = 1.4
# 스케일 단계마다 수행할 경사 상승법 횟수
iterations = 20

# 손실이 10보다 커지면 이상한 그림이 되는 것을 
# 피하기 위해 경사 상승법 과정을 중지합니다.
max_loss = 10.

# 사용할 이미지 경로를 씁니다.
base_image_path = '../data/original_photo_deep_dream.jpg'

# 기본 이미지를 넘파이 배열로 로드합니다.
# (이 함수는 코드 8-13에 정의되어 있습니다)
img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
# 경사 상승법을 시행할 스케일 크기를 정의한
# 튜플의 리스트를 준비합니다.
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
    for dim in original_shape])
    successive_shapes.append(shape)

# 이 리스트를 크기 순으로 뒤집습니다.
successive_shapes = successive_shapes[::-1]

# 이미지의 넘파이 배열을 가장 작은 스케일로 변경합니다.
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('처리할 이미지 크기', shape)
    # 딥드림 이미지의 스케일을 키웁니다.
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          # 경사 상승법을 실행하고 이미지를 변경합니다.
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    # 작게 줄인 원본 이미지의 스케일을 높입니다. 픽셀 경계가 보일 것입니다.
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    # 이 크기에 해당하는 원본 이미지의 고해상도 버전을 계산합니다.
    same_size_original = resize_img(original_img, shape)
    # 이 두 이미지의 차이가 스케일을 높였을 때 손실된 디테일입니다.
    lost_detail = same_size_original - upscaled_shrunk_original_img
    # 손실된 디테일을 딥드림 이미지에 다시 주입합니다.
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='../data/final_dream.png')

* 이 코드는 다음에 나오는 유틸리티 함수를 사용합니다. 넘파이 배열 기반의 함수이며 이름으로 역할을 알 수 있습니다. 이 함수를 사용하려면 싸이파이를 설치해야 합니다.

In [ ]:
# 유틸리티 함수(Utility Functions)
import scipy
from tensorflow.keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)


def preprocess_image(image_path):
    # 사진을 열고 크기를 줄이고 인셉션 V3가 인식하는
    # 텐서 포맷으로 변환하는 유틸리티 함수
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # 넘파이 배열을 적절한 이미지 포맷으로 변환하는 유틸리티 함수
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        # inception_v3.preprocess_input 함수에서 수행한 전처리 과정을 복원합니다
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# 원본과 딥드림 코드 실행 결과 출력하기
# print the original and the result of DeepDream code execution
plt.imshow(plt.imread(base_image_path))
plt.figure()

plt.imshow(deprocess_image(np.copy(img)))
plt.show()

* 샌프란시스코 만과 구글 캠퍼스 사이 작은 언덕에서 위와 같은 딥드림 이미지를 얻었습니다.
* 손실로 사용할 층을 바꾸어 보면서 어떤 일이 발생하는지 꼭 시도해 보세요! 네트워크의 하위 층은 지역적이고 비교적 덜 추상적인 표현을 가지고 있기 때문에 딥드림 이미지에 기하학적 패턴이 많이 생깁니다. 상위 층은 강아지 눈, 새의 깃털처럼 ImageNet에 많이 등장하는 물체를 기반으로 뚜렷이 구분되는 시각 패턴을 만듭니다. layer_contributions 딕셔너리의 파라미터를 랜덤하게 생성하여 여러 가지 층 조합을 빠르게 탐색할 수 있습니다.

### 8.2.2 정리
* 딥드림은 네트워크가 학습한 표현을 기반으로 컨브넷을 거꾸로 실행하여 입력 이미지를 생성합니다.
* 재미있는 결과가 만들어지고, 때로는 환각제 때문에 시야가 몽롱해진 사람이 만든 이미지 같기도 합니다.
* 이 과정은 이미지 모델이나 컨브넷에 국한되지 않습니다. 음성, 음악 등에도 적용될 수 있습니다.

## 8.3 뉴럴 스타일 트랜스퍼(Neural Style Transfer)

* 딥드림 이외에 딥러닝을 사용하여 이미지를 변경하는 또 다른 주요 분야는 뉴럴 스타일 트랜스퍼(neural style transfer)입니다. 2015년 여름 리온 게티스(Leon Gatys) 등이 소개했습니다. 뉴럴 스타일 트랜스퍼 알고리즘은 처음 소개된 이후에 많이 개선되었고 여러 변종들이 생겼습니다. 스마트폰의 사진 앱에도 쓰입니다. 이 절에서는 간단하게 원본 논문에 소개한 방식에 집중하겠습니다.

* 뉴럴 스타일 트랜스퍼는 타깃 이미지의 콘텐츠를 보존하면서 참조 이미지의 스타일을 타깃 이미지에 적용합니다.

![style transfer](https://s3.amazonaws.com/book.keras.io/img/ch8/style_transfer.png)

* 여기에서 스타일은 질감, 색깔, 이미지에 있는 다양한 크기의 시각 요소를 의미합니다. 콘텐츠는 이미지에 있는 고수준의 대형 구조를 말합니다. 예를 들어 빈센트 반 고흐의 별이 빛나는 밤에서 파랑과 노랑색의 원을 그리는 듯한 붓질을 하나의 스타일로 생각할 수 있습니다. 튀빙겐 사진의 건물은 콘텐츠로 생각할 수 있습니다.

* 텍스처 생성과 밀접하게 연관된 스타일 트랜스퍼의 아이디어는 2015년 뉴럴 스타일 트랜스퍼가 개발되기 이전에 이미 이미지 처리 분야에서 오랜 역사를 가지고 있습니다. 딥러닝을 기반으로 한 스타일 트랜스퍼 구현은 고전적인 컴퓨터 비전 기법으로 만든 것과는 비견할 수 없는 결과를 제공합니다. 창조적인 컴퓨터 비전 애플리케이션 분야에 새로운 르네상스를 열었습니다.

* 스타일 트랜스퍼 구현 이면에 있는 핵심 개념은 모든 딥러닝 알고리즘의 핵심과 동일합니다. 목표를 표현한 손실 함수를 정의하고 이 손실을 최소화합니다. 여기서 원하는 것은 다음과 같습니다. 참조 이미지의 스타일을 적용하면서 원본 이미지의 콘텐츠를 보존하는 것입니다. 콘텐츠와 스타일을 수학적으로 정의할 수 있다면 최소화할 손실 함수는 다음과 같을 것입니다.

```python
loss = distance(style(reference_image) - style(generated_image)) +
       distance(content(original_image) - content(generated_image))
```

* 여기에서 `distance`는 L2 노름 같은 노름 함수입니다. `content` 함수는 이미지의 콘텐츠 표현을 계산합니다. `style` 함수는 이미지의 스타일 표현을 계산합니다.

* 이 손실을 최소화하면 `style(generated_image)`는 `style(reference_image)`와 가까워지고 `content(generated_image)`는 `content(original_image)`와 가까워집니다. 앞서 정의한 스타일 트랜스퍼의 목적을 달성할 수 있습니다.

* 게티스 등은 심층 합성곱 신경망을 사용해 `style`과 `content` 함수를 수학적으로 정의할 수 있다는 것을 보였습니다. 어떻게 하는 것인지 알아 보죠.

### 8.3.1 콘텐츠 손실(Content Loss)

* 앞서 배웠듯이 네트워크에 있는 하위 층의 활성화는 이미지에 관한 국부적인 정보를 담고 있습니다. 반면 상위 층의 활성화일수록 점점 전역적이고 추상적인 정보를 담게 됩니다. 다른 방식으로 생각하면 컨브넷 층의 활성화는 이미지를 다른 크기의 콘텐츠로 분해한다고 볼 수 있습니다. 컨브넷 상위 층의 표현을 사용하면 전역적이고 추상적인 이미지 콘텐츠를 찾을 것입니다.

* 타깃 이미지와 생성된 이미지를 사전 훈련된 컨브넷에 주입하여 상위 층의 활성화를 계산합니다. 이 두 값 사이의 L2 노름이 콘텐츠 손실로 사용하기에 좋습니다. 상위 층에서 보았을 때 생성된 이미지와 원본 타깃 이미지를 비슷하게 만들 것입니다. 컨브넷의 상위 층이 보는 것이 입력 이미지의 콘텐츠라고 가정하면 이미지의 콘텐츠를 보존하는 방법으로 사용할 수 있습니다.

### 8.3.2 스타일 손실

* 콘텐츠 손실은 하나의 상위 층만 사용합니다. 게티스 등이 정의한 스타일 손실은 컨브넷의 여러 층을 사용합니다. 하나의 스타일이 아니라 참조 이미지에서 컨브넷이 추출한 모든 크기의 스타일을 잡아야 합니다.

* 게티스 등은 층의 활성화 출력의 **그람 행렬**(Gram matrix)을 스타일 손실로 사용했습니다. 그람 행렬은 층의 특성 맵들의 내적(inner product)입니다. 내적은 층의 특성 사이에 있는 상관관계를 표현한다고 이해할 수 있습니다. 이런 특성의 상관관계는 특정 크기의 공간적인 패턴 통계를 잡아 냅니다. 경험적으로 봤을 때 이 층에서 찾은 텍스처에 대응됩니다.

* 스타일 참조 이미지와 생성된 이미지로 층의 활성화를 계산합니다. 스타일 손실은 그 안에 내재된 상관관계를 비슷하게 보존하는 것이 목적입니다. 결국 스타일 참조 이미지와 생성된 이미지에서 여러 크기의 텍스처가 비슷하게 보이도록 만듭니다.

★ **요약**

* 요약하면 사전 훈련된 컨브넷을 사용해 다음과 같은 손실을 정의할 수 있습니다.

    * 콘텐츠를 보존하기 위해 타깃 콘텐츠 이미지와 생성된 이미지 사이에서 상위 층의 활성화를 비슷하게 유지합니다. 이 컨브넷은 타깃 이미지와 생성된 이미지에서 동일한 것을 보아야 합니다.
    * 스타일을 보존하기 위해 저수준 층과 고수준 층에서 활성화 내에 상관관계를 비슷하게 유지합니다. 특성의 상관관계는 텍스처를 잡아냅니다. 생성된 이미지와 스타일 참조 이미지는 여러 크기의 텍스처를 공유할 것입니다.

* 이제 2015년 뉴럴 스타일 트랜스퍼 원본 알고리즘을 케라스로 구현해 보죠. 잠시 후 알게 되겠지만 이전 절에서 만든 딥드림 구현과 공통점이 많습니다.

### 8.3.3 케라스에서 뉴럴 스타일 트랜스퍼 구현하기

* 뉴럴 스타일 트랜스퍼는 사전 훈련된 컨브넷 중 어떤 것을 사용해서도 구현할 수 있습니다. 여기에서는 게티스 등이 사용한 VGG19 네트워크를 사용하겠습니다. VGG19는 5장에서 소개한 VGG16 네트워크의 변종으로 합성곱 층이 3개 더 추가되었습니다.

* 일반적인 과정은 다음과 같습니다:

    1. 스타일 참조 이미지, 타깃 이미지, 생성된 이미지를 위해 VGG19의 층 활성화를 동시에 계산하는 네트워크를 설정합니다.
    2. 세 이미지에서 계산한 층 활성화를 사용하여 앞서 설명한 손실 함수를 정의합니다. 이 손실을 최소화하여 스타일 트랜스퍼를 구현할 것입니다.
    3. 손실 함수를 최소화할 경사 하강법 과정을 설정합니다.

* 스타일 참조 이미지와 타깃 이미지의 경로를 정의하는 것부터 시작하죠. 처리할 이미지는 크기가 비슷합니다(크기가 많이 다르면 스타일 트랜스퍼를 구현하는 것이 더 어렵습니다). 모두 높이가 400 픽셀이 되도록 크기를 변경하겠습니다.

In [ ]:
# 변수 초깃값 정의하기
# Defining variable initial values
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img

# 변환하려는 이미지 경로
target_image_path = '../data/portrait.png'
# 스타일 이미지 경로
style_reference_image_path = '../data/popova.jpg'

# 생성된 사진의 차원
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

* VGG19 컨브넷에 입출력할 이미지의 로드, 전처리, 사후 처리를 위해 유틸리티 함수를 정의합니다.

In [ ]:
# 유틸리티 함수(Utility Functions)
import numpy as np
from tensorflow.keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # ImageNet의 평균 픽셀 값을 더합니다
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

* VGG19 네트워크를 설정해 보죠. 스타일 참조 이미지, 타깃 이미지 그리고 생성된 이미지가 담긴 플레이스홀더로 이루어진 배치를 입력으로 받습니다. 플레이스홀더는 심볼릭 텐서로 넘파이 배열로 밖에서 값을 제공해야 합니다. 스타일 참조 이미지와 타깃 이미지는 이미 준비된 데이터이므로 K.constant를 사용해 정의합니다. 반면 플레이스홀더에 담길 생성된 이미지는 계속 바뀝니다.

In [ ]:
# 사전 훈련된 VGG19 네크워크를 로딩하고 3개 이미지에 적용하기
# Loading pre-trained VGG19 network and applying it to three images
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# 생성된 이미지를 담을 플레이스홀더
combination_image = K.placeholder((1, img_height, img_width, 3))

# 세 개의 이미지를 하나의 배치로 합칩니다
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# 세 이미지의 배치를 입력으로 받는 VGG 네트워크를 만듭니다.
# 이 모델은 사전 훈련된 ImageNet 가중치를 로드합니다
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('모델 로드 완료.')

* 콘텐츠 손실을 정의해 보죠. VGG19 컨브넷의 상위 층은 타깃 이미지와 생성된 이미지를 동일하게 바라봐야 합니다: